In [2]:
import requests
import bs4
import pyttsx3

In [4]:
engine = pyttsx3.init()
engine.say("Hello rodney")
engine.runAndWait()

In [16]:
engine = pyttsx3.init()
engine.say(task)
engine.runAndWait()

In [1]:
from vosk import Model, KaldiRecognizer
import pyaudio
import json
import os
import time

model = Model("C:/Users/Home/Documents/Downloads/vosk-model-en-us-0.22-lgraph/vosk-model-en-us-0.22-lgraph")
recognizer = KaldiRecognizer(model, 16000)

mic = pyaudio.PyAudio()
stream = mic.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8192)
stream.start_stream()

print("Listening offline with Vosk...")

#while True:
    #data = stream.read(4096, exception_on_overflow=False)
    #if recognizer.AcceptWaveform(data):
        #result = json.loads(recognizer.Result())
        #print("You said:", result['text'])

try:
    while True:
        data = stream.read(2048, exception_on_overflow=False)
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            print("You said:", result['text'])
        time.sleep(0.01)
except KeyboardInterrupt:
    print("\nStopped by user.")
    stream.stop_stream()
    stream.close()
    p.terminate()


Listening offline with Vosk...

Stopped by user.


NameError: name 'p' is not defined

In [6]:
import pvporcupine
import struct
import os
import json
import time
import pyttsx3
import pyaudio
from vosk import Model, KaldiRecognizer

model_path = "C:/Users/Home/Documents/Downloads/vosk-model-en-us-0.22-lgraph/vosk-model-en-us-0.22-lgraph"
model = Model(model_path)
recognizer = KaldiRecognizer(model, 16000)

engine = pyttsx3.init()
engine.setProperty('rate', 160)
engine.setProperty('volume', 1.0)

DATA_FILE = "jarvis_data.json"
if not os.path.exists(DATA_FILE):
    with open(DATA_FILE, "w") as f:
        json.dump({"ideas": [], "reminders": []}, f)
        
porcupine = pvporcupine.create(keywords=["jarvis"])
pa = pyaudio.PyAudio()
stream = pa.open(
    rate=porcupine.sample_rate,
    channels=1,
    format=pyaudio.paInt16,
    input=True,
    frames_per_buffer=porcupine.frame_length
)

#p = pyaudio.PyAudio()
#stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000,
                #input=True, frames_per_buffer=2048)
#stream.start_stream()

def wait_for_wake_word():
    speak("Say 'Hey Jarvis' when you need me.")
    while True:
        pcm = stream.read(porcupine.frame_length, exception_on_overflow=False)
        pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)
        if porcupine.process(pcm) >= 0:
            speak("I'm listening...")
            listen_for_command()
            speak("Anything else? Say 'Hey Jarvis' again if you need me.")

def listen_for_command():
    recog = KaldiRecognizer(model, 16000)
    local_stream = pa.open(format=pyaudio.paInt16, channels=1, rate=16000,
                           input=True, frames_per_buffer=2048)
    local_stream.start_stream()
    end_time = time.time() + 8  # listen for 8 seconds
    while time.time() < end_time:
        data = local_stream.read(2048, exception_on_overflow=False)
        if recog.AcceptWaveform(data):
            result = json.loads(recog.Result())
            text = result.get("text", "")
            if text:
                print("You said:", text)
                if parse_command(text) == "exit":
                    local_stream.stop_stream()
                    local_stream.close()
                    stream.stop_stream()
                    porcupine.delete()
                    pa.terminate()
                    exit()
    local_stream.stop_stream()
    local_stream.close()

def speak(text):
    print("JARVIS:", text)
    engine.say(text)
    engine.runAndWait()

def save_data(intent, content):
    with open(DATA_FILE, "r+") as f:
        data = json.load(f)
        data[intent].append(content)
        f.seek(0)
        json.dump(data, f, indent=2)

def parse_command(text):
    text = text.lower()
    if "idea" in text:
        idea = text.replace("add idea", "").strip()
        save_data("ideas", idea)
        speak("Noted your idea. Great thinking!")
    elif "remind me" in text:
        reminder = text.replace("remind me to", "").strip()
        save_data("reminders", reminder)
        speak(f"I'll remind you to {reminder}.")
    elif "stop" in text or "exit" in text:
        speak("Goodbye, stay productive!")
        return "exit"
    else:
        speak("Sorry, I didn't catch a command. Try again?")
    return "continue"

speak("Hello! I'm your assistant. What can I help you with today?")

try:
    while True:
        data = stream.read(2048, exception_on_overflow=False)
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            text = result.get("text", "")
            if text:
                print("You said:", text)
                status = parse_command(text)
                if status == "exit":
                    break
                speak("Anything else you'd like to add?")
except KeyboardInterrupt:
    speak("Session ended. Goodbye.")
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


TypeError: create() missing 1 required positional argument: 'access_key'

In [1]:
import os
import json
import time
import struct
import pyttsx3
import pyaudio
from vosk import Model, KaldiRecognizer
import pvporcupine

# === SETUP ===
# Load Vosk model
VOSK_MODEL_PATH = "C:/Users/Home/Documents/Downloads/vosk-model-en-us-0.22-lgraph/vosk-model-en-us-0.22-lgraph"
model = Model(VOSK_MODEL_PATH)

# Setup TTS
engine = pyttsx3.init()
engine.setProperty('rate', 160)
engine.setProperty('volume', 1.0)

# Wake word setup
key = os.getenv("ACCESS_KEY")
porcupine = pvporcupine.create(key, keywords=["jarvis"])

pa = pyaudio.PyAudio()
wake_stream = pa.open(
    rate=porcupine.sample_rate,
    channels=1,
    format=pyaudio.paInt16,
    input=True,
    frames_per_buffer=porcupine.frame_length
)

# Data file
DATA_FILE = "jarvis_data.json"
if not os.path.exists(DATA_FILE):
    with open(DATA_FILE, "w") as f:
        json.dump({"ideas": [], "reminders": []}, f)

# === FUNCTIONS ===
def speak(text):
    print("JARVIS:", text)
    engine.say(text)
    engine.runAndWait()

def save_data(intent, content):
    with open(DATA_FILE, "r+") as f:
        data = json.load(f)
        data[intent].append(content)
        f.seek(0)
        json.dump(data, f, indent=2)

def parse_command(text):
    text = text.lower()
    if "idea" in text:
        idea = text.replace("add idea", "").strip()
        save_data("ideas", idea)
        speak("Great idea. I've noted that.")
    elif "remind me" in text:
        reminder = text.replace("remind me to", "").strip()
        save_data("reminders", reminder)
        speak(f"Okay, I will remind you to {reminder}.")
    elif "exit" in text or "stop" in text:
        speak("Goodbye. Stay productive!")
        return "exit"
    else:
        speak("I didn't catch that. Could you repeat it?")
    return "continue"

def listen_for_command():
    recog = KaldiRecognizer(model, 16000)
    stream = pa.open(format=pyaudio.paInt16, channels=1, rate=16000,
                     input=True, frames_per_buffer=2048)
    stream.start_stream()

    speak("I'm listening...")
    end_time = time.time() + 8  # listen for 8 seconds

    while time.time() < end_time:
        data = stream.read(2048, exception_on_overflow=False)
        if recog.AcceptWaveform(data):
            result = json.loads(recog.Result())
            text = result.get("text", "")
            if text:
                print("You said:", text)
                if parse_command(text) == "exit":
                    stream.stop_stream()
                    stream.close()
                    return "exit"
    stream.stop_stream()
    stream.close()

def wait_for_wake_word():
    speak("Say 'Hey Jarvis' when you need me.")
    while True:
        pcm = wake_stream.read(porcupine.frame_length, exception_on_overflow=False)
        pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)
        if porcupine.process(pcm) >= 0:
            result = listen_for_command()
            if result == "exit":
                break
            speak("Anything else? Just say 'Hey Jarvis'.")

# === START ===
try:
    wait_for_wake_word()
except KeyboardInterrupt:
    speak("Session ended. Goodbye.")
finally:
    wake_stream.stop_stream()
    wake_stream.close()
    porcupine.delete()
    pa.terminate()


JARVIS: Say 'Hey Jarvis' when you need me.
JARVIS: I'm listening...
JARVIS: Anything else? Just say 'Hey Jarvis'.
JARVIS: I'm listening...
You said: but
JARVIS: I didn't catch that. Could you repeat it?
JARVIS: Anything else? Just say 'Hey Jarvis'.
JARVIS: I'm listening...
You said: stop
JARVIS: Goodbye. Stay productive!


In [1]:
import ollama

response = ollama.chat(
    model="llama3",
    messages=[
        {"role": "user", "content": "Hello Jarvis, who are you?"}
    ]
)

print(response['message']['content'])

ResponseError: model "llama3" not found, try pulling it first (status code: 404)